# Post Processing

This notebook is used to post process submissions. It's divided into 5 majors steps :
1. Extract automatically all archives from the input directory.
2. Convert all denses point cloud files (`.las` or `.laz`) into DEMs with `point2dem` command from ASP.
3. Coregister DEMs with references DEMs. And compute difference between coregistered DEMs and references DEMs.
4. Compute some global statistics on all the Post Processing.
5. Generate some plots.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import history.postprocessing as pp
from prefect.task_runners import ConcurrentTaskRunner

/home/godinlu/micromamba/envs/history/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Settings and Paths Managing

For the post Preprocessing workflow their is a lot of directory and paths. To simplify it, let's used the class `history.postprocessing.PathsManager`. 

In [4]:
raw_dir = "/mnt/summer/USERS/DEHECQA/history/output/raw"
extracted_dir = "/mnt/summer/USERS/DEHECQA/history/output/extracted"
processing_dir = "/mnt/summer/USERS/DEHECQA/history/output/new_processing" 
plots_dir = "/mnt/summer/USERS/DEHECQA/history/output/plots" 

# other settings
OVERWRITE = False
DRY_RUN = False # set this to True to avoid process
MAX_WORKERS = 4
VERBOSE = True
PDAL_EXEC_PATH = "/home/godinlu/micromamba/envs/pdal/bin/pdal"

task_runner = ConcurrentTaskRunner(max_workers=MAX_WORKERS)

In [ ]:
%%capture
pp.uncompress_all_submissions.with_options(task_runner=task_runner)(raw_dir, extracted_dir, OVERWRITE)

In [8]:
pp.create_pointcloud_symlinks(extracted_dir, processing_dir)

 casa_grande - aerial → 20 pointcloud(s) linked.
 casa_grande - kh9mc → 6 pointcloud(s) linked.
 casa_grande - kh9pc → 1 pointcloud(s) linked.
 iceland - aerial → 9 pointcloud(s) linked.
 iceland - kh9mc → 4 pointcloud(s) linked.
 iceland - kh9pc → 0 pointcloud(s) linked.


In [20]:
%%capture

history.postprocessing.batch.run_postprocessing.with_options(task_runner=task_runner)(extracted_dir, processing_dir, PDAL_EXEC_PATH, dry_run=DRY_RUN)


In [22]:
%%capture

history.postprocessing.batch.generate_postprocessing_plots.with_options(task_runner=task_runner)(processing_dir, plots_dir)